# Generate All Paper Figures

This notebook generates all high-quality figures for the paper.

Figures generated:
1. Basic Π(x), B(x), E(x) curves
2. Error growth in log-log scale
3. Judge comparison
4. Spectrum analysis
5. Zero distribution
6. Complexity distribution

All figures are saved to `../output/figures/` for LaTeX inclusion.


In [ ]:
import sys
import os
from pathlib import Path
from datetime import datetime

# Robust path setup
def setup_paths():
    current_dir = Path(os.getcwd())
    if current_dir.name == 'notebooks':
        simulation_dir = str(current_dir.parent)
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    elif current_dir.name == 'simulation':
        simulation_dir = str(current_dir)
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    for parent in current_dir.parents:
        if parent.name == 'simulation':
            simulation_dir = str(parent)
            if simulation_dir not in sys.path:
                sys.path.insert(0, simulation_dir)
            return simulation_dir
    for path in ['..', '../simulation', 'simulation']:
        abs_path = os.path.abspath(path)
        if abs_path not in sys.path:
            sys.path.insert(0, abs_path)
    return None

setup_paths()

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display, HTML

from core.action_space import generate_world
from core.judgement_system import BiasedJudge, NoisyJudge, ConservativeJudge, batch_evaluate
from core.ethical_primes import select_ethical_primes, compute_Pi_and_error, analyze_error_growth, compare_error_distributions
from analysis.zeta_function import build_m_sequence, compute_spectrum, find_approximate_zeros, analyze_spectrum_peaks
from visualization.plots import *

setup_paper_style()
np.random.seed(42)


Generating all paper figures...
All figures generated successfully!
Output location: ../output/figures/


## Configuration

Set parameters for figure generation: style, format, resolution, and batch options.


In [ ]:
# Configuration
config = {
    'num_actions': 2000,
    'tau': 0.3,
    'X_max': 100,
    'X_max_zeta': 200,
    'random_seed': 42,
    'figure_style': 'paper',  # 'paper', 'presentation', 'poster'
    'figure_format': 'pdf',   # 'pdf', 'png', 'svg'
    'dpi': 300,
    'output_dir': '../output/figures',
    'batch_mode': False,      # Generate for multiple seeds
    'batch_seeds': [42, 123, 456] if False else [42]
}

# Ensure output directory exists
os.makedirs(config['output_dir'], exist_ok=True)

print("Figure Generation Configuration:")
print("=" * 60)
for key, value in config.items():
    print(f"  {key}: {value}")
print("=" * 60)


## Generate All Figures

Generate all paper figures with the specified configuration.


In [ ]:
print("Generating all paper figures...")
print("=" * 60)

# Generate data
actions = generate_world(num_actions=config['num_actions'], 
                        complexity_dist='zipf', 
                        random_seed=config['random_seed'])
judges = {
    'Biased': BiasedJudge(bias_strength=0.2),
    'Noisy': NoisyJudge(noise_scale=0.3),
    'Conservative': ConservativeJudge(threshold=0.5)
}
results = batch_evaluate(actions, judges, tau=config['tau'])

generated_figures = {}

# Figure 1: Basic Pi, B, E curves (Biased Judge)
print("Generating Figure 1: Pi, B, E curves...")
primes_biased = select_ethical_primes(results['Biased'])
Pi_x, B_x, E_x, x_vals = compute_Pi_and_error(primes_biased, X_max=config['X_max'])
fig1_path = f"{config['output_dir']}/paper_fig1_pi_b_e.{config['figure_format']}"
plot_Pi_B_E(x_vals, Pi_x, B_x, E_x, save_path=fig1_path, show=False)
generated_figures['fig1'] = fig1_path
print(f"  Saved: {fig1_path}")

# Figure 2: Error growth analysis
print("Generating Figure 2: Error growth...")
analysis = analyze_error_growth(E_x, x_vals)
fig2_path = f"{config['output_dir']}/paper_fig2_error_growth.{config['figure_format']}"
plot_error_growth(x_vals, E_x, analysis, save_path=fig2_path, show=False)
generated_figures['fig2'] = fig2_path
print(f"  Saved: {fig2_path}")

# Figure 3: Multi-judge comparison
print("Generating Figure 3: Judge comparison...")
comparison = compare_error_distributions(results, X_max=config['X_max'])
fig3_path = f"{config['output_dir']}/paper_fig3_judge_comparison.{config['figure_format']}"
plot_multi_judge_errors(comparison, save_path=fig3_path, show=False)
generated_figures['fig3'] = fig3_path
print(f"  Saved: {fig3_path}")

# Figure 4: Spectrum
print("Generating Figure 4: Spectrum...")
m = build_m_sequence(primes_biased, X_max=config['X_max_zeta'])
freqs, amps = compute_spectrum(m)
peaks = analyze_spectrum_peaks(freqs, amps)
fig4_path = f"{config['output_dir']}/paper_fig4_spectrum.{config['figure_format']}"
plot_spectrum(freqs, amps, peaks, save_path=fig4_path, show=False)
generated_figures['fig4'] = fig4_path
print(f"  Saved: {fig4_path}")

# Figure 5: Zero distribution
print("Generating Figure 5: Zero distribution...")
zeros = find_approximate_zeros(m, grid_size=40)
fig5_path = f"{config['output_dir']}/paper_fig5_zeros.{config['figure_format']}"
plot_zero_distribution(zeros, save_path=fig5_path, show=False)
generated_figures['fig5'] = fig5_path
print(f"  Saved: {fig5_path}")

# Figure 6: Complexity distribution
print("Generating Figure 6: Complexity distribution...")
fig6_path = f"{config['output_dir']}/paper_fig6_complexity_dist.{config['figure_format']}"
plot_complexity_distribution(actions, save_path=fig6_path, show=False)
generated_figures['fig6'] = fig6_path
print(f"  Saved: {fig6_path}")

print("\n" + "=" * 60)
print(f"All {len(generated_figures)} figures generated successfully!")
print(f"Output location: {config['output_dir']}")


## Figure Validation

Check that all generated figures exist and meet quality requirements.


In [ ]:
# Validate generated figures
print("Validating generated figures...")
print("=" * 60)

validation_results = {}
min_size_kb = 10  # Minimum file size in KB

for fig_name, fig_path in generated_figures.items():
    if os.path.exists(fig_path):
        file_size = os.path.getsize(fig_path) / 1024  # Size in KB
        is_valid = file_size >= min_size_kb
        validation_results[fig_name] = {
            'exists': True,
            'size_kb': file_size,
            'valid': is_valid,
            'path': fig_path
        }
        status = "[OK]" if is_valid else "[WARNING]"
        print(f"{status} {fig_name}: {file_size:.1f} KB - {fig_path}")
    else:
        validation_results[fig_name] = {
            'exists': False,
            'size_kb': 0,
            'valid': False,
            'path': fig_path
        }
        print(f"[FAIL] {fig_name}: File not found - {fig_path}")

# Summary
valid_count = sum(1 for v in validation_results.values() if v['valid'])
total_count = len(validation_results)
print("\n" + "=" * 60)
print(f"Validation Summary: {valid_count}/{total_count} figures valid")
if valid_count == total_count:
    print("All figures passed validation!")
else:
    print(f"Warning: {total_count - valid_count} figures failed validation")


## Figure Preview Gallery

Display all generated figures for visual inspection.


In [ ]:
# Display figure previews
print("Figure Preview Gallery:")
print("=" * 60)

# Convert PDF to PNG for preview if needed, or just show paths
for fig_name, fig_path in generated_figures.items():
    if os.path.exists(fig_path):
        print(f"\n{fig_name.upper()}: {os.path.basename(fig_path)}")
        print(f"  Path: {fig_path}")
        print(f"  Size: {os.path.getsize(fig_path) / 1024:.1f} KB")
        
        # Try to display if PNG
        if fig_path.endswith('.png'):
            try:
                display(Image(fig_path))
            except:
                print("  (Preview not available)")
        elif fig_path.endswith('.pdf'):
            print("  (PDF preview not available in notebook - open file to view)")
    else:
        print(f"\n{fig_name.upper()}: FILE NOT FOUND")


## Batch Processing

Generate figures for multiple random seeds or parameter sets.


In [ ]:
# Batch processing (if enabled)
if config['batch_mode'] and len(config['batch_seeds']) > 1:
    print("Batch Processing Mode:")
    print("=" * 60)
    print(f"Generating figures for {len(config['batch_seeds'])} random seeds...")
    
    batch_results = {}
    for seed in config['batch_seeds']:
        print(f"\nProcessing seed {seed}...")
        # Generate data with different seed
        actions_batch = generate_world(num_actions=config['num_actions'], 
                                      complexity_dist='zipf', 
                                      random_seed=seed)
        results_batch = batch_evaluate(actions_batch, judges, tau=config['tau'])
        
        # Generate figures with seed suffix
        seed_suffix = f"_seed{seed}"
        batch_results[seed] = {}
        
        # Figure 1
        primes_batch = select_ethical_primes(results_batch['Biased'])
        Pi_x_b, B_x_b, E_x_b, x_vals_b = compute_Pi_and_error(primes_batch, X_max=config['X_max'])
        fig1_batch = f"{config['output_dir']}/paper_fig1_pi_b_e{seed_suffix}.{config['figure_format']}"
        plot_Pi_B_E(x_vals_b, Pi_x_b, B_x_b, E_x_b, save_path=fig1_batch, show=False)
        batch_results[seed]['fig1'] = fig1_batch
        
        print(f"  Generated figures for seed {seed}")
    
    print(f"\nBatch processing complete! Generated figures for {len(config['batch_seeds'])} seeds.")
else:
    print("Batch processing disabled (batch_mode=False or single seed).")


## LaTeX Integration

Generate LaTeX code for including figures in the paper.


In [ ]:
# Generate LaTeX code for figure inclusion
latex_captions = {
    'fig1': 'Distribution functions $\\Pi(x)$, $B(x)$, and $E(x)$ for the Biased Judge.',
    'fig2': 'Error growth analysis showing $|E(x)|$ vs. complexity $x$ in log-log scale.',
    'fig3': 'Comparison of error growth across different judgment systems.',
    'fig4': 'Frequency spectrum of the ethical zeta function.',
    'fig5': 'Distribution of zeros of the ethical zeta function in the complex plane.',
    'fig6': 'Distribution of action complexities in the generated moral action space.'
}

latex_code = []
latex_code.append("% Auto-generated LaTeX code for figure inclusion")
latex_code.append(f"% Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
latex_code.append("")

for fig_name, fig_path in generated_figures.items():
    if os.path.exists(fig_path):
        # Get relative path from LaTeX file location (assuming figures/ directory)
        rel_path = os.path.basename(fig_path)
        caption = latex_captions.get(fig_name, f"Figure {fig_name}")
        
        latex_code.append(f"\\begin{{figure}}[htbp]")
        latex_code.append(f"  \\centering")
        if config['figure_format'] == 'pdf':
            latex_code.append(f"  \\includegraphics[width=0.8\\textwidth]{{figures/{rel_path}}}")
        else:
            latex_code.append(f"  \\includegraphics[width=0.8\\textwidth]{{figures/{rel_path}}}")
        latex_code.append(f"  \\caption{{{caption}}}")
        latex_code.append(f"  \\label{{fig:{fig_name}}}")
        latex_code.append(f"\\end{{figure}}")
        latex_code.append("")

# Save LaTeX code
latex_output_path = '../output/figures_latex_code.tex'
with open(latex_output_path, 'w') as f:
    f.write('\n'.join(latex_code))

print("Generated LaTeX code:")
print("=" * 60)
print('\n'.join(latex_code[:20]))  # Show first 20 lines
if len(latex_code) > 20:
    print(f"\n... (total {len(latex_code)} lines)")
print(f"\nFull LaTeX code saved to: {latex_output_path}")

# Also generate subfigure layout option
subfigure_code = []
subfigure_code.append("% Alternative: Subfigure layout for side-by-side comparison")
subfigure_code.append("\\begin{figure}[htbp]")
subfigure_code.append("  \\centering")
subfigure_code.append("  \\begin{subfigure}[b]{0.48\\textwidth}")
subfigure_code.append("    \\centering")
subfigure_code.append("    \\includegraphics[width=\\textwidth]{figures/paper_fig1_pi_b_e.pdf}")
subfigure_code.append("    \\caption{$\\Pi(x)$, $B(x)$, $E(x)$}")
subfigure_code.append("    \\label{fig:sub1}")
subfigure_code.append("  \\end{subfigure}")
subfigure_code.append("  \\hfill")
subfigure_code.append("  \\begin{subfigure}[b]{0.48\\textwidth}")
subfigure_code.append("    \\centering")
subfigure_code.append("    \\includegraphics[width=\\textwidth]{figures/paper_fig2_error_growth.pdf}")
subfigure_code.append("    \\caption{Error growth}")
subfigure_code.append("    \\label{fig:sub2}")
subfigure_code.append("  \\end{subfigure}")
subfigure_code.append("  \\caption{Main results}")
subfigure_code.append("  \\label{fig:combined}")
subfigure_code.append("\\end{figure}")

subfigure_path = '../output/figures_subfigure_layout.tex'
with open(subfigure_path, 'w') as f:
    f.write('\n'.join(subfigure_code))

print(f"\nSubfigure layout code saved to: {subfigure_path}")


## Export Metadata

Save generation parameters and metadata for reproducibility.


In [ ]:
# Export metadata
import json

metadata = {
    'generation_timestamp': datetime.now().isoformat(),
    'configuration': config,
    'generated_figures': {
        fig_name: {
            'path': fig_path,
            'exists': os.path.exists(fig_path),
            'size_kb': os.path.getsize(fig_path) / 1024 if os.path.exists(fig_path) else 0,
            'caption': latex_captions.get(fig_name, '')
        }
        for fig_name, fig_path in generated_figures.items()
    },
    'validation': {
        'total_figures': len(generated_figures),
        'valid_figures': sum(1 for v in validation_results.values() if v['valid']),
        'validation_results': validation_results
    }
}

metadata_path = '../output/figure_generation_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata exported to: {metadata_path}")
print(f"\nSummary:")
print(f"  Generated: {len(generated_figures)} figures")
print(f"  Valid: {sum(1 for v in validation_results.values() if v['valid'])}/{len(validation_results)}")
print(f"  Total size: {sum(v['size_kb'] for v in validation_results.values()):.1f} KB")
print(f"  Output directory: {config['output_dir']}")
